# 前情提要
此次作業交的 CSV 檔為 TF-IDF + Logistic Regression 的結果  
在 Kaggle 的成績為 0.70750

In [ ]:
# 要預先載入
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

[nltk 教學](https://clay-atlas.com/blog/2019/07/30/%E3%80%90nlp%E3%80%91nltk-%E5%9F%BA%E6%9C%AC%E6%95%99%E5%AD%B8/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
corpus_root = 'drive/My Drive/DS HW/' 

df = pd.read_csv(corpus_root+'train.csv')
df_test = pd.read_csv(corpus_root+'test_no_answer.csv')

## 前處理

### 刪除標點符號  

ref: https://gist.github.com/youngsoul/34d4ded8d4edb82fdfb8f9a91ade2c0d  

    table = str.maketrans({key: None for key in string.punctuation})
    return text.lower().translate(table)
ref: https://www.kite.com/python/answers/how-to-remove-all-punctuation-marks-with-nltk-in-python  

ref: http://blog.fantasy.codes/python/2013/07/02/py-re-module/


In [ ]:
import re
import string

# 還原縮寫 主要還原 not
abbr = {        
    "isn t ":"is not ",
    "wasn t ":"was not ",
    "aren t ":"are not ",
    "weren t ":"were not ",
    "can t ":"can not ",
    "couldn t ":"could not ",
    "don t ":"do not ",
    "doesn t ":"does not ",
    "didn t ":"did not ",
    "won t ":"will not ",
    "wouldn t ":"would not ",
    "hasn t ":"has not ",
    "haven t ":"have not ",
    "hadn t ":"had not ",
    "shouldn t ":"should not ",
}

def remove_punctuation(text):
    tokenizer = nltk.RegexpTokenizer(r'\w+')
    new_words = tokenizer.tokenize(text)
    new_words = ' '.join(new_words)
    
    for word, origin in abbr.items():
        new_words = new_words.replace(word, origin)

    return new_words

### 詞型還原  

ref: https://www.zmonster.me/2016/01/21/lemmatization-survey.html





In [ ]:
from nltk.corpus import wordnet, stopwords
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else :
        return  None

def lemmatize_sentence(sentence):
    res = []
    lemmatizer = WordNetLemmatizer()
    for word, pos in pos_tag(word_tokenize(sentence)):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res.append(lemmatizer.lemmatize(word, pos=wordnet_pos))

    return ' '.join(res)

df['TEXT'] = df['TEXT'].apply(remove_punctuation)
df['TEXT'] = df['TEXT'].apply(lemmatize_sentence)
df_test['TEXT'] = df_test['TEXT'].apply(remove_punctuation)
df_test['TEXT'] = df_test['TEXT'].apply(lemmatize_sentence)
print(df['TEXT'])

## Word2Vec

In [ ]:
from gensim.models import KeyedVectors
from smart_open import open as smart_open
embeddings = KeyedVectors.load_word2vec_format(corpus_root + 'GoogleNews-vectors-negative300.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
def w2v(text):
    docs_vectors = pd.DataFrame()
    for doc in text:
        temp = pd.DataFrame()
        for word in doc.split(' '):
            try:
                word_vec = embeddings[word]
                temp = temp.append(pd.Series(word_vec), ignore_index = True)
            except:
                pass
        doc_vector = temp.mean()
        docs_vectors = docs_vectors.append(doc_vector, ignore_index = True)
    print(docs_vectors.shape)
    
    return docs_vectors

In [ ]:
w2v_all = w2v(df['TEXT'])

(11000, 300)


## LSTM

- https://machinelearningmastery.com/use-word-embedding-layers-deep-learning-keras/
- https://machinelearningmastery.com/how-to-develop-lstm-models-for-time-series-forecasting/
- http://frankchen.xyz/2017/12/18/How-to-Use-Word-Embedding-Layers-for-Deep-Learning-with-Keras/

In [ ]:
import keras
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import BatchNormalization
from keras.utils import to_categorical, np_utils
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, Activation, SpatialDropout1D, Dropout

# 參數設定
max_features = 50000
maxlen = 25
batch_size = 16
classes = 2

tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(df['TEXT'])
sequences_train = tokenizer.texts_to_sequences(df['TEXT'])

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(sequences_train, maxlen=maxlen)

y_train = np.array(df["LABEL"]-1)

Y_train = np_utils.to_categorical(y_train, classes)

print('Build model...')
LSTM_model = Sequential()
LSTM_model.add(Embedding(max_features, 256, input_length=maxlen))
LSTM_model.add(SpatialDropout1D(0.25))
LSTM_model.add(LSTM(256, dropout=0.3, recurrent_dropout=0.4))
LSTM_model.add(BatchNormalization())
LSTM_model.add(Dense(256 , activation='relu'))
LSTM_model.add(Dropout(0.3))
LSTM_model.add(Dense(128 , activation='relu'))
LSTM_model.add(Dropout(0.4))
LSTM_model.add(Dense(64 , activation='relu'))
LSTM_model.add(Dropout(0.5))
LSTM_model.add(Dense(classes , activation='softmax'))

optimizer = keras.optimizers.Adam(lr=1e-3)
LSTM_model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

print('Train...')
LSTM_model.fit(X_train, Y_train, batch_size=batch_size, epochs=5)

print("Generating test predictions...")
sequences_test = tokenizer.texts_to_sequences(df_test['TEXT'])
X_test = sequence.pad_sequences(sequences_test, maxlen=maxlen)
LSTM_pred_test = LSTM_model.predict_classes(X_test, verbose=0)

Pad sequences (samples x time)
Build model...
Train...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/5
11000/11000 [==============================] - 62s 6ms/step - loss: 0.6729 - accuracy: 0.5895
Epoch 2/5
11000/11000 [==============================] - 61s 6ms/step - loss: 0.5548 - accuracy: 0.7300
Epoch 3/5
11000/11000 [==============================] - 62s 6ms/step - loss: 0.4789 - accuracy: 0.7881
Epoch 4/5
11000/11000 [==============================] - 61s 6ms/step - loss: 0.4175 - accuracy: 0.8207
Epoch 5/5
11000/11000 [==============================] - 62s 6ms/step - loss: 0.3621 - accuracy: 0.8469
Generating test predictions...


## 輸出檔案
LSTM

In [ ]:
LSTMsubmission = pd.DataFrame({'row_id': df_test['row_id'],
                           'LABEL': LSTM_pred_test})
LSTMsubmission.to_csv(corpus_root + 'Submission_LSTM.csv', index = False)
LSTMsubmission

,row_id,LABEL
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
1995,1995,1
1996,1996,1
1997,1997,0
1998,1998,1


In [ ]:
a = submission[submission['LABEL'] == 1]
len(a)